In [0]:
import pandas as pd

from battery_optimise import *
from algo_utils import plot_actions

pd.set_option('display.float_format',lambda x : '%.2f' % x)

### Simulate battery optimisation

In [0]:
# Read data
data = pd.read_csv("../data/energy_prices_fy24.csv")
data['time'] = pd.to_datetime(data['time'])

In [0]:
result = battery_optimisation(data.time, data.energy_price, cycle_cost=1000.0, daily_cycle_limit=1, solver='appsi_highs')

In [0]:
result.to_csv('../result/result_dispatch_cycle_cost_fy24_v2.csv')

##### Visualise result

In [0]:
# chart_data = pd.read_csv('test_result.csv', index_col='time', parse_dates=['time'])

In [0]:
start_day = 90
days_to_plot = 5
start = int(288*(start_day-1))
end = int(288*(start_day+days_to_plot-1))
plot_actions(result.spot_price, result.power, opening_capacity=result.opening_capacity, start=start, end=end)

##### Assumptions:


- **Perfect foresight** on spot price <br>
- Battery does not degrade throughout the optimisation horizon <br>
- Battery cannot charge and discharge concurrently 

To formulate the algorithm, we've treated the battery system as a linear programming problem. The following are the objective function, parameters, decision variables and operational constraint for the linear programming model. The objective function is the objective we would like to maximise, which is the revenue over the time horizon of optimisation. <br>
- The objective function:<br>
$\sum_{t=1}^{T}{P_t}(\frac{D_t{E}}{2}{\eta}-\frac{C_t}{2{\eta}})$ <br>
<br>
- Parameters: <br>
$T$: Time horizon of arbitrage <br>
$P_t$: Spot price at time t <br>
$E$: Charge and discharge efficiency of the battery <br>
$\eta$: Marginal Loss Factor <br>
<br>
- Decision Variables: <br>
$Capacity_t$: Opening Capacity at time t <br>
$C_t$: Charge Power at time t <br>
$D_t$: Discharge Power at time t <br>
<br>
- Opertaion Constraint: <br>
$Capacity_t = {Capacity_{t-1}} + \frac{C_t}{2}E - \frac{D_t}{2}$ <br>
$0 \leq {Capacity_t} \leq 580, \forall t \in T$ <br>
$0 \leq {C_t} \leq 300, \forall t \in T$ <br>
$0 \leq {D_t} \leq 300, \forall t \in T$ <br>
$D_t \leq \frac{Capacity_t\times{E}}{2}$ <br>
$when\; P_t < 0,\:D_t = 0$ <br>